[website](https://boardgamegeek.com/boardgame/224517/brass-birmingham/stats)

In [67]:
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    StaleElementReferenceException,
    WebDriverException,
    InvalidSessionIdException
)
import time
from tqdm import tqdm
import os
import json
import pandas as pd

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

allboardgames = 0
all_games_links = []

page_one = 1 
page_end = 3

try:
    for page in range(page_one, page_end):
        url = f"https://boardgamegeek.com/browse/boardgame/page/{page}"
        driver.get(url)
        time.sleep(3)

        try:
            game_links_per_page = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[@class="primary"]')]
            all_games_links.extend(game_links_per_page)
        except (NoSuchElementException, StaleElementReferenceException) as e:
                print(f"Error on page {page}: {e}")
except (TimeoutException, WebDriverException, InvalidSessionIdException) as e:
    print(f"Navigation error: {e}")

game_links = 'GoToGames.txt'
with open(game_links, 'w') as file:
    for game_link in all_games_links:
         file.write(game_link + '\n')

print(f"The links to all {len(all_games_links)} board games has been collected and stored successfully")

with tqdm(total=len(all_games_links), desc="Games Scraped") as pbar:
     for href in all_games_links:
          driver.get(href)
          boardgames = []

          time.sleep(10)

          page_source = driver.page_source
          soup = BeautifulSoup(page_source, 'html.parser')

          SeeFullCredits = WebDriverWait(driver,10).until(
               EC.element_to_be_clickable((By.XPATH, '//a[@ui-sref="geekitem.credits"]'))
          )
          SeeFullCredits.click()

In [90]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://boardgamegeek.com/boardgame/224517/brass-birmingham/credits"
driver.get(url)

spans = driver.find_elements(By.XPATH, "//span[@class='ng-binding ng-scope']")

try:
    minimum_players_root = spans[0].text.strip()
    minimum_players = int(minimum_players_root)

    maximum_players_root = spans[1].text.split('–')[1]
    maximum_players = int(maximum_players_root)

    minimum_playing_time_root = spans[3].text.strip()
    minimum_playing_time = int(minimum_playing_time_root)

    maximum_playing_time_root = spans[4].text.split('–')[1]
    maximum_playing_time = int(maximum_playing_time_root)

    game_name = spans[6].text.strip()
    release_year = int(spans[7].text)
except Exception as e:
    print("Error scraping credits")

In [98]:
print(minimum_playing_time, maximum_playing_time, minimum_players, maximum_players, game_name, release_year )

60 120 2 4 Brass: Birmingham 2018
